<a href="https://colab.research.google.com/github/kiseonjeong/keras-ml/blob/fc-ae/AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>